<a href="https://colab.research.google.com/github/biggymart/Group_proj/blob/master/fasterRCNN_helmet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **동빈나 YouTube**
**RPN** : ConvNet -> GPU, faster than CPU, only discerns whether an object exists

2015년도 기준으로 **5FPS** 정도면 아주 빠른 속도였음

keywords: Objectness scores, A pyramid of anchors, Alternating training, feature sharing, Translation invariant, NMS (IOU 0.7, 2000 proposals)

Architecture: Two modules: (1) RPN (2) fast RCNN

k boxes -> cls 2k, reg 4k

IOU 0.7 이상: positive; 0.3 이하: negative;

PASCAL VOC 2007 (5k train; 5k test) -> 59.9%; Ablation Experiment

07 + 12 -> 70%, two-stage

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

In [ ]:
if torch.cuda.is_available():
  device = torh.device('cuda')
  print(device, torch.cuda.get_device_name(0))
else:
  device = torch.device('cpu')
  print(device)

cpu


Read a batch of trainingdimages along with their bounding boxes and labels.
In this example, I only read 1 image. (i.e batch_size = 1)

In [ ]:
# input image could be of any size
img0 = cv2.imread('helmet_9.jpg')
img0 = cv2.cvtColor(img0, COLOR_BGR2RGB)
print(img0.shape)
plt.imshow(img0)
plt.show()

NameError: ignored

In [ ]:
# Object information: a set of bounding boxes [ymin, xmin, ymax, xmax] and their labels
bbox0 = np.array([160, 147, 260, 234], [139, 312, 200, 348])
labels = np.array([1, 1]) # 0: background, 1: helmet

In [ ]:
# display bounding box and labels
# cv2.putText(TextYouWantToPrint)
img0_clone = np.copy(img0)
for i in range(len(bbox0)):
  cv2.rectangle(img0_clone, (bbox0[i][1], bbox0[i][0]), (bbox0[i][3], bbox0[i][2]), color=(0, 255, 0), thickness=3)
  cv2.putText(img0_clone, str(int(labels[i])), (bbox0[i][3], bbox0[i][2]), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,255), thickness=3)
plt.imshow(img0_clone)
plt.show()

NameError: ignored

Resize the input images to (h=800, w=800)

In [ ]:
img = cv2.resize(img0, dsize=(800, 800), interpolation=cv2.INTER_CUBIC)
plt.imshow(img)
plt.show()

In [ ]:
# change the bounding box coordinates
Wratio = 800/img0.shape[1]
Hratio = 800/img0.shape[0]
ratioLst = [Hratio, Wratio, Hratio, Wratio]
bbox = []
for box in bbox0:
  box = [int(a * b) for a, b in zip(box, ratioLst)]
  bbox.append(box)
bbox = np.array(bbox)
print(bbox)

In [ ]:
# display bounding box and labels
img_clone = np.copy(img)
bbox_clone = bbox.astype(int)
for i in range(len(bbox)):
  # Draw rectangle
  cv2.rectangle(img_clone, (bbox[i][1], bbox[i][0]), (bbox[i][3], bbox[i][2]), color=(0, 255, 0), thickness=3)
  # Write the prediction
  cv2.putText(img_clone, str(int(labels[i])), (bbox[i][3], bbox[i][2]), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,255), thickness=3)
plt.imshow(img_clone)
plt.show()

Use VGG16 to extract features from input images
Input images (batch_size, H=800, W=800, d=3), Features: (batch_size, H=50, W=50, d=512)

In [ ]:
# List all the layers of VGG16
model = torchvision.models.vgg16(pretrained=True).to(device)
fc = list(model.features)
print(len(fc))

In [ ]:
# Collect layers with output feature map size (W, H) < 50
# test image array [1, 3, 800, 800]
dummy_img = torch.zeros((1, 3, 800, 800)).float()
print(dummy_img)

req_features = []
k = dummy_img.clone().to(device)
for i in fc:
  k = i(k)
  if k.size()[2] < 800//16: # 800/16 = 50
    break
  req_features.append(i)
  out_channels = k.size()[1]
print(len(req_features)) # 30
print(out_channels) # 512

In [ ]:
# Convert this list into a Sequential module
faster_rcnn_fc_extractor = nn.Sequential(*req_features)

In [ ]:
transform = transforms.Compose([transforms.ToTensor()]) # Define PyTorch Transform
imgTensor = transform(img).to(device)
imgTensor = imgTensor.unsqueeze(0)
out_map = faster_rcnn_fc_extractor(imgTensor)
print(out_map.size())
# torch.Size([1, 512, 50, 50])

In [ ]:
# visualize the first 5 channels of the 50*50*512 feature maps
imgArray = out_map.cpu().numpy().squeeze(0)
fig = plt.figure(figsize=(12, 4))
figNo = 1
for i in range(5):
  fig.add_subplot(1, 5, figNo)
  plt.imshow(imgArray[i], cmap='gray')
  figNo += 1
plt.show()

Generate 22,500 anchor boxes on each input image
50x50=2500 anchors, each anchor generates 9 anchor boxes, Total = 50x50x9=22,500

In [ ]:
# x, y intervals to generate anchor box center
fc_size = (800//16)
ctr_x = np.arange(16, (fc_size+1)*16, 16)
ctr_y = np.arange(16, (fc_size+1)*16, 16)
print(len(ctr_x), ctr_x)

In [ ]:
# coordinates of the 2500 center points to generate anchor boxes
index = 0
ctr = np.zeros((2500,2))
for x in range(len(ctr_x)):
  for y in range(len(ctr_y)):
    ctr[index, 1] = ctr_x[x] - 8
    ctr[index, 0] = ctr_y[x] - 8
    index += 1
print(ctr.shape)

In [ ]:
# display the 2500 anchors
img_clone = np.copy(img)
plt.figure(figsize=(9,6))
for i in range(ctr.shape[0]):
  cv2.circle(img_clone, (int(ctr[i][0]), int(ctr[i][1])), radius=1, color=(255,0,0), thickness=1)
plt.imshow(img_clone)
plt.show()

In [ ]:
# for each of the 2500 anchors, generate 9 anchor boxes
# 2500 * 9 = 25,000 anchor boxes
ratios = [0.5, 1, 2]
scales = [8, 16, 32]
sub_samples = 16
anchor_boxes = np.zeros(((fc_size * fc_size * 9), 4))
index = 0
for c in ctr:
  ctr_y, ctr_x = c
  for i in range(len(ratios)):
    for j in range(len(scales)):
      h = sub_sample * scales[j] * np.sqrt(ratios[i])
      w = sub_sample * scales[j] * np.sqrt(1./ ratios[i])
      anchor_boxes[index, 0] = ctr_y - h /2.
      anchor_boxes[index, 1] = ctr_x - w /2.
      anchor_boxes[index, 2] = ctr_y - h /2.
      anchor_boxes[index, 3] = ctr_x - w /2.
      index += 1
print(anchor_boxes.shape)